## 🔐 Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login
```

or

```bash
az login --use-device-code
```

# 📋 Loan Underwriting Agent with Azure AI Search

This notebook demonstrates how to use **Azure AI Search with Agentic Mode** for RAG (Retrieval Augmented Generation) in a **Loan Underwriting** scenario.

## Features Covered:
- Setting up `AzureAISearchContextProvider` with **agentic mode**
- Creating a Knowledge Base for underwriting policy documents
- Multi-hop reasoning across underwriting guidelines and criteria
- Intelligent query planning for complex eligibility questions

### Industry Use Case: Loan Underwriting & Risk Assessment

Our agent will help underwriters and loan officers:
- Review underwriting guidelines and approval criteria
- Analyze debt-to-income (DTI) and loan-to-value (LTV) requirements
- Research documentation requirements for different loan types
- Synthesize eligibility criteria across multiple loan products

### ⚠️ Important Disclaimer ⚠️
> **This notebook is for educational purposes only. All underwriting criteria is sample data for training purposes. Always follow your institution's official underwriting guidelines and regulatory requirements.**

### 🔍 Why Agentic Mode?

| Feature | Agentic Mode | Semantic Mode |
|---------|-------------|---------------|
| Query Planning | ✅ Multi-hop reasoning | ❌ Single query |
| Knowledge Bases | ✅ Required | ❌ Uses index directly |
| Accuracy | Higher (36% improvement) | Good for simple queries |
| Speed | Slightly slower | Faster |
| Best For | Complex questions | Simple lookups |

## Prerequisites

Before running this notebook, ensure you have:

1. **Azure AI Search Service**: With a search index or knowledge base
2. **Microsoft Foundry Project**: With a deployed model (gpt-4o recommended)
3. **Authentication**: Azure CLI installed and authenticated
4. **Environment Variables** in root `.env` file:
   - `AI_FOUNDRY_PROJECT_ENDPOINT`
   - `AZURE_AI_MODEL_DEPLOYMENT_NAME`
   - `AZURE_AI_SEARCH_ENDPOINT`
   - `AZURE_AI_SEARCH_API_KEY` (optional if using managed identity)
   - `AZURE_SEARCH_INDEX_NAME` (for auto-creating Knowledge Base)
   - `AZURE_OPENAI_ENDPOINT` (required for agentic mode with index)

If you need to use a different tenant:
```bash
az login --tenant <tenant-id>
```

## Import Libraries

Import the required libraries for Azure AI Search context provider:

In [ ]:
import os
from pathlib import Path

from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient, AzureAISearchContextProvider
from azure.identity.aio import AzureCliCredential
from dotenv import load_dotenv

# Load environment variables from root .env (force override)
load_dotenv('../../.env', override=True)

# Verify environment setup
required_vars = [
    'AI_FOUNDRY_PROJECT_ENDPOINT',
    'AZURE_AI_MODEL_DEPLOYMENT_NAME',
    'AZURE_AI_SEARCH_ENDPOINT',
]

missing = [var for var in required_vars if not os.getenv(var)]
if missing:
    print(f"⚠️ Missing environment variables: {missing}")
    print("Please configure these in your root .env file")
else:
    print("🔧 Environment Configuration:")
    print(f"✅ Project Endpoint: {os.getenv('AI_FOUNDRY_PROJECT_ENDPOINT')[:50]}...")
    print(f"✅ Model Deployment: {os.getenv('AZURE_AI_MODEL_DEPLOYMENT_NAME')}")
    print(f"✅ Search Endpoint: {os.getenv('AZURE_AI_SEARCH_ENDPOINT')}")
    print(f"✅ Search Index: {os.getenv('AZURE_SEARCH_INDEX_NAME', 'Not set - will need KB name')}")

## Configuration 📋

Set up the configuration for Azure AI Search and the agent. We'll use agentic mode for intelligent multi-hop retrieval.

In [ ]:
# Azure AI Search configuration
SEARCH_ENDPOINT = os.environ["AZURE_AI_SEARCH_ENDPOINT"]
SEARCH_API_KEY = os.environ.get("AZURE_AI_SEARCH_API_KEY")  # Optional if using managed identity

# Microsoft Foundry configuration
PROJECT_ENDPOINT = os.environ["AI_FOUNDRY_PROJECT_ENDPOINT"]
MODEL_DEPLOYMENT = os.environ.get("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o")

# For agentic mode, we need either:
# Option 1: An existing Knowledge Base name
KNOWLEDGE_BASE_NAME = os.environ.get("AZURE_SEARCH_KNOWLEDGE_BASE_NAME")

# Option 2: Auto-create KB from an index (requires OpenAI endpoint)
INDEX_NAME = os.environ.get("AZURE_SEARCH_INDEX_NAME")
AZURE_OPENAI_RESOURCE_URL = os.environ.get("AZURE_OPENAI_ENDPOINT")

print("📊 Search Configuration:")
print(f"  Endpoint: {SEARCH_ENDPOINT}")
print(f"  API Key: {'Configured' if SEARCH_API_KEY else 'Using managed identity'}")
print(f"  Knowledge Base: {KNOWLEDGE_BASE_NAME or 'Will auto-create from index'}")
print(f"  Index Name: {INDEX_NAME or 'Not configured'}")
print(f"  OpenAI Resource: {AZURE_OPENAI_RESOURCE_URL or 'Not configured'}")

## Define Underwriting Research Queries 📋

These sample queries demonstrate the loan underwriting use case. The agentic mode excels at:
- Complex multi-hop queries requiring synthesis across policy documents
- Questions that need reasoning about eligibility criteria relationships
- Analysis requests that span multiple loan product guidelines

## 🔧 Create Index and Upload Sample Data

Run this cell **once** to create the `underwriting-index` and upload sample loan underwriting documents. This cell will:
1. Create the Azure AI Search index with the proper schema
2. Upload sample underwriting policy documents
3. Verify the data was uploaded successfully

> **Note**: Skip this cell if the index already exists with data.

In [ ]:
import time
from openai import AzureOpenAI
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex, SearchField, SearchFieldDataType, VectorSearch,
    HnswAlgorithmConfiguration, VectorSearchProfile, AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters, SemanticSearch, SemanticConfiguration,
    SemanticPrioritizedFields, SemanticField
)
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import ResourceNotFoundError

# ============================================================================
# CONFIGURATION
# ============================================================================
index_name = "underwriting-index"
knowledge_source_name = f"{index_name}-source"
knowledge_base_name = f"{index_name}-kb"
embedding_deployment = "text-embedding-3-large"

# Get Azure OpenAI endpoint (remove trailing slash if present)
azure_openai_endpoint = AZURE_OPENAI_RESOURCE_URL.rstrip("/")

# Create credentials and clients
credential = AzureKeyCredential(SEARCH_API_KEY)
index_client = SearchIndexClient(endpoint=SEARCH_ENDPOINT, credential=credential)
aoai_client = AzureOpenAI(
    azure_endpoint=azure_openai_endpoint,
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version="2024-06-01"
)

# ============================================================================
# SAMPLE UNDERWRITING DATA
# ============================================================================
UNDERWRITING_DOCUMENTS = [
    {
        "id": "uw-001",
        "title": "Conventional Loan Underwriting Guidelines",
        "content": """Conventional Loan Underwriting Criteria:
        
1. Credit Score Requirements:
- Minimum credit score: 620 for most conventional loans
- For best rates: 740+ credit score recommended
- Scores between 620-680 may require additional documentation

2. Debt-to-Income (DTI) Ratio:
- Maximum front-end DTI: 28% (housing expenses to income)
- Maximum back-end DTI: 36-43% (total debt to income)
- DTI up to 50% may be approved with compensating factors

3. Loan-to-Value (LTV) Ratio:
- Primary residence: Up to 97% LTV with PMI
- Second home: Maximum 90% LTV
- Investment property: Maximum 80% LTV

4. Down Payment Requirements:
- Minimum 3% down for first-time homebuyers
- 5% down for repeat buyers
- 20% down to avoid PMI

5. Reserve Requirements:
- Primary residence: 2 months reserves
- Second home: 2-6 months reserves
- Investment property: 6 months reserves"""
    },
    {
        "id": "uw-002", 
        "title": "FHA Loan Underwriting Guidelines",
        "content": """FHA Loan Underwriting Criteria:

1. Credit Score Requirements:
- Minimum 580 credit score for 3.5% down payment
- 500-579 credit score requires 10% down payment
- No minimum credit score for manual underwriting

2. Debt-to-Income (DTI) Ratio:
- Front-end DTI: Maximum 31%
- Back-end DTI: Maximum 43%
- Higher DTI up to 50% with compensating factors

3. Loan-to-Value (LTV) Ratio:
- Maximum 96.5% LTV with 3.5% down
- Maximum 90% LTV for 500-579 credit scores

4. Mortgage Insurance Premium (MIP):
- Upfront MIP: 1.75% of loan amount
- Annual MIP: 0.45% to 1.05% depending on LTV and term
- MIP required for life of loan if LTV > 90%

5. Property Requirements:
- Must be primary residence
- Property must meet FHA minimum property standards
- Appraisal valid for 120 days"""
    },
    {
        "id": "uw-003",
        "title": "Income Verification Requirements",
        "content": """Income Verification Documentation Requirements:

1. W-2 Employees:
- Last 2 years W-2 forms
- Most recent 30 days pay stubs
- Verbal Verification of Employment (VVOE)
- Employment must be stable (2+ years same employer or field)

2. Self-Employed Borrowers:
- Last 2 years personal tax returns (all schedules)
- Last 2 years business tax returns (if applicable)
- Year-to-date profit and loss statement
- Business license verification
- CPA letter may be required

3. Other Income Sources:
- Social Security: Award letter + 2 months bank statements
- Pension/Retirement: Award letter + 1099-R
- Rental Income: Lease agreements + 2 years tax returns
- Alimony/Child Support: Court order + 12 months receipt history

4. Asset Documentation:
- Last 2 months bank statements (all pages)
- Investment account statements
- Gift letter for gift funds (with donor bank statements)
- Large deposits must be sourced and documented"""
    },
    {
        "id": "uw-004",
        "title": "Jumbo Loan Underwriting Guidelines",
        "content": """Jumbo Loan Underwriting Criteria:

1. Loan Limits (2024):
- Exceeds conforming loan limit of $766,550
- High-cost areas: Exceeds $1,149,825

2. Credit Score Requirements:
- Minimum credit score: 700-720
- Preferred credit score: 740+
- More stringent credit history review

3. Debt-to-Income (DTI) Ratio:
- Maximum DTI: 43% (stricter than conventional)
- Some lenders allow up to 45% with strong reserves

4. Down Payment Requirements:
- Minimum 10-20% down payment
- 25-30% down for investment properties
- No PMI options available

5. Reserve Requirements:
- 6-12 months reserves required
- Higher reserves for multiple properties
- Liquid assets preferred

6. Additional Requirements:
- Two appraisals may be required
- More extensive income documentation
- May require additional underwriter review"""
    },
    {
        "id": "uw-005",
        "title": "VA Loan Underwriting Guidelines", 
        "content": """VA Loan Underwriting Criteria:

1. Eligibility Requirements:
- Active duty: 90 consecutive days during wartime
- Active duty: 181 days during peacetime
- National Guard/Reserves: 6 years of service
- Certificate of Eligibility (COE) required

2. Credit Requirements:
- No official minimum credit score
- Most lenders require 580-620 minimum
- Bankruptcy: 2 years from discharge (Chapter 7)
- Foreclosure: 2 years from completion

3. Debt-to-Income (DTI) Ratio:
- Maximum DTI: 41% guideline
- Higher DTI acceptable with residual income
- Residual income requirements vary by region and family size

4. Loan-to-Value (LTV):
- 100% LTV allowed (no down payment required)
- No PMI required
- VA Funding Fee: 1.25% to 3.3% (may be financed)

5. Property Requirements:
- Must be primary residence
- Must meet VA Minimum Property Requirements (MPRs)
- VA appraisal required"""
    },
    {
        "id": "uw-006",
        "title": "Credit History and Derogatory Events",
        "content": """Credit History Guidelines and Waiting Periods:

1. Bankruptcy:
- Chapter 7: 4 years (Conventional), 2 years (FHA/VA)
- Chapter 13: 2 years from discharge, 4 years from dismissal
- Must demonstrate re-established credit

2. Foreclosure:
- Conventional: 7 years from completion
- FHA: 3 years from completion
- VA: 2 years from completion
- Extenuating circumstances may reduce waiting period

3. Short Sale/Deed-in-Lieu:
- Conventional: 4 years (2 years with extenuating circumstances)
- FHA: 3 years
- VA: 2 years

4. Late Payments:
- Mortgage lates in past 12 months: Generally not allowed
- 30-day late: May be acceptable with explanation
- 60+ day late: Typically requires 12 months clean payment history

5. Collections and Charge-offs:
- Medical collections: Generally excluded
- Non-medical collections > $2,000: May need to be paid
- Judgments: Must be paid or in payment plan"""
    },
    {
        "id": "uw-007",
        "title": "Property Appraisal Requirements",
        "content": """Property Appraisal Requirements:

1. Appraisal Standards:
- Must be performed by licensed/certified appraiser
- Must comply with USPAP standards
- Must use appropriate comparable sales (within 1 year, 1 mile)

2. Conventional Loan Appraisals:
- Appraisal waiver (PIW) available for certain loans
- Desktop appraisal option for lower risk transactions
- Full appraisal for higher LTV or complex properties

3. FHA Appraisal Requirements:
- FHA roster appraiser required
- Property must meet Minimum Property Requirements (MPRs)
- Health and safety issues must be addressed
- Appraisal valid for 120 days (extendable to 240)

4. Condition Requirements:
- Roof: Minimum 2-3 years remaining life
- HVAC: Must be functional
- Electrical/Plumbing: Must be safe and functional
- Foundation: No significant cracks or settlement

5. Value Discrepancies:
- If appraised value < purchase price: Renegotiate or increase down payment
- Reconsideration of Value (ROV) process available
- Second appraisal may be ordered for significant discrepancies"""
    },
    {
        "id": "uw-008",
        "title": "Investment Property Underwriting",
        "content": """Investment Property Underwriting Guidelines:

1. Down Payment Requirements:
- Single-family: Minimum 15-20% down
- 2-4 units: Minimum 25% down
- No PMI options for investment properties

2. Credit Score Requirements:
- Minimum 620 (most lenders require 680+)
- Higher rates for scores below 720

3. Reserve Requirements:
- 6 months PITI reserves for subject property
- 2 months reserves for each additional financed property
- Up to 10 financed properties allowed

4. Rental Income Calculation:
- 75% of gross rental income used (25% vacancy factor)
- Lease agreement required for occupied properties
- Appraiser's market rent for vacant properties
- Schedule E from tax returns for existing rentals

5. DTI Considerations:
- Full PITI added to debt obligations
- Rental income offset allowed per guidelines
- Maximum DTI typically 45%

6. Additional Requirements:
- Property management experience preferred
- May require landlord insurance verification
- Higher interest rates than primary residence"""
    }
]

# ============================================================================
# CLEANUP: Delete knowledge base, knowledge source, then index (in order)
# ============================================================================
print("🧹 Cleaning up existing resources...")
print("=" * 60)

# Step 1: Delete knowledge base first (it references knowledge sources)
try:
    index_client.get_knowledge_base(knowledge_base_name)
    print(f"🗑️ Deleting knowledge base '{knowledge_base_name}'...")
    index_client.delete_knowledge_base(knowledge_base_name)
    time.sleep(2)
    print(f"   ✅ Knowledge base deleted")
except ResourceNotFoundError:
    print(f"   ℹ️ Knowledge base '{knowledge_base_name}' not found (OK)")
except Exception as e:
    print(f"   ⚠️ Could not delete knowledge base: {e}")

# Step 2: Delete knowledge source (it references the index)
try:
    index_client.get_knowledge_source(knowledge_source_name)
    print(f"🗑️ Deleting knowledge source '{knowledge_source_name}'...")
    index_client.delete_knowledge_source(knowledge_source_name)
    time.sleep(1)
    print(f"   ✅ Knowledge source deleted")
except ResourceNotFoundError:
    print(f"   ℹ️ Knowledge source '{knowledge_source_name}' not found (OK)")
except Exception as e:
    print(f"   ⚠️ Could not delete knowledge source: {e}")

print(f"\n{'='*60}")
print("✅ Cleanup complete - ready to create index")

# ============================================================================
# FUNCTIONS
# ============================================================================

def create_search_index(index_name):
    """Create a vector search index with hybrid search capabilities."""
    
    # Check if index exists and delete it
    try:
        index_client.get_index(index_name)
        print(f"   🗑️ Deleting existing index '{index_name}'...")
        index_client.delete_index(index_name)
        time.sleep(2)
    except ResourceNotFoundError:
        pass
    
    # Define vector search configuration
    vector_search = VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(name="hnsw-config")
        ],
        profiles=[
            VectorSearchProfile(
                name="vector-profile",
                algorithm_configuration_name="hnsw-config",
                vectorizer_name="openai-vectorizer"
            )
        ],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="openai-vectorizer",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=azure_openai_endpoint,
                    deployment_name=embedding_deployment,
                    model_name=embedding_deployment
                )
            )
        ]
    )
    
    # Define semantic search configuration
    semantic_search = SemanticSearch(
        default_configuration_name="semantic-config",
        configurations=[
            SemanticConfiguration(
                name="semantic-config",
                prioritized_fields=SemanticPrioritizedFields(
                    content_fields=[SemanticField(field_name="content")]
                )
            )
        ]
    )
    
    # Define index fields
    fields = [
        SearchField(name="id", type=SearchFieldDataType.String, key=True),
        SearchField(name="title", type=SearchFieldDataType.String, searchable=True),
        SearchField(name="content", type=SearchFieldDataType.String, searchable=True),
        SearchField(
            name="content_vector",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            searchable=True,
            vector_search_dimensions=3072,
            vector_search_profile_name="vector-profile"
        )
    ]
    
    # Create the index with vector and semantic search
    index = SearchIndex(
        name=index_name,
        fields=fields,
        vector_search=vector_search,
        semantic_search=semantic_search
    )
    
    index_client.create_or_update_index(index)


def generate_embeddings(texts, batch_size=10):
    """Generate embeddings using Azure OpenAI."""
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        response = aoai_client.embeddings.create(
            input=batch,
            model=embedding_deployment
        )
        embeddings.extend([item.embedding for item in response.data])
    return embeddings


def upload_documents_to_index(index_name, documents):
    """Upload documents with embeddings to search index."""
    search_client = SearchClient(
        endpoint=SEARCH_ENDPOINT,
        index_name=index_name,
        credential=credential
    )
    
    # Generate embeddings for all documents
    texts = [doc["content"] for doc in documents]
    embeddings = generate_embeddings(texts)
    
    # Add embeddings to documents
    for i, doc in enumerate(documents):
        doc["content_vector"] = embeddings[i]
    
    # Upload documents
    search_client.upload_documents(documents)
    return len(documents)


# ============================================================================
# EXECUTE: CREATE INDEX AND UPLOAD DOCUMENTS
# ============================================================================
print("\n🏗️ Creating Azure AI Search Index with Vector Search...")
print("=" * 60)

print(f"\n📚 Creating: {index_name}")
create_search_index(index_name)
print(f"   ✅ Index created successfully")

print("\n🧠 Generating embeddings and uploading documents...")
print("=" * 60)

print(f"\n📤 Processing: {index_name}")
print(f"   ⏳ Generating {len(UNDERWRITING_DOCUMENTS)} embeddings...")
count = upload_documents_to_index(index_name, UNDERWRITING_DOCUMENTS)
print(f"   ✅ Uploaded {count} documents")

# Wait for indexing
time.sleep(2)

# Verify
search_client = SearchClient(endpoint=SEARCH_ENDPOINT, index_name=index_name, credential=credential)
doc_count = search_client.get_document_count()

print(f"\n{'=' * 60}")
print(f"✅ Index setup complete!")
print(f"   • Index name: {index_name}")
print(f"   • Document count: {doc_count}")
print(f"   • Embedding model: {embedding_deployment}")
print(f"   • Vector dimensions: 3072")
print(f"   • Semantic config: semantic-config")

In [ ]:
# FSI Loan Underwriting sample queries
UNDERWRITING_QUERIES = [
    # Policy overview query
    "What are the key underwriting criteria for mortgage loan approval?",
    
    # Risk assessment query  
    "What debt-to-income (DTI) ratios are acceptable for different loan types?",
    
    # Documentation requirements
    "What documentation is required for income verification in loan applications?",
    
    # Eligibility criteria
    "What credit score requirements apply to conventional vs FHA loans?",
]

print("📋 Sample Loan Underwriting Research Queries:")
for i, query in enumerate(UNDERWRITING_QUERIES, 1):
    print(f"  {i}. {query[:80]}{'...' if len(query) > 80 else ''}")

## Create Search Context Provider 🔍

The `AzureAISearchContextProvider` in **agentic mode** uses Knowledge Bases for intelligent query planning and multi-hop retrieval.

**Key Parameters:**
- `mode="agentic"`: Enables multi-hop reasoning
- `knowledge_base_name`: Use existing KB (recommended)
- `index_name` + `azure_openai_resource_url`: Auto-create KB from index
- `knowledge_base_output_mode`: `"extractive_data"` or `"answer_synthesis"`
- `retrieval_reasoning_effort`: `"minimal"`, `"low"`, or `"medium"`

> **Note**: For this notebook, we'll use the `underwriting-index` which contains loan underwriting policies, eligibility criteria, and documentation requirements.

In [ ]:
def create_search_provider():
    """Create Azure AI Search context provider with agentic mode."""
    
    print("🔍 Creating Azure AI Search Context Provider (Agentic Mode)...")
    print("   This mode uses Knowledge Bases for intelligent query planning.\n")
    
    # Configure based on available settings
    if KNOWLEDGE_BASE_NAME:
        # Option 1: Use existing Knowledge Base (recommended)
        print(f"✅ Using existing Knowledge Base: {KNOWLEDGE_BASE_NAME}")
        return AzureAISearchContextProvider(
            endpoint=SEARCH_ENDPOINT,
            api_key=SEARCH_API_KEY,
            credential=AzureCliCredential() if not SEARCH_API_KEY else None,
            mode="agentic",
            knowledge_base_name=KNOWLEDGE_BASE_NAME,
            # Retrieval configuration
            knowledge_base_output_mode="extractive_data",  # or "answer_synthesis"
            retrieval_reasoning_effort="minimal",  # or "low", "medium"
        )
    elif INDEX_NAME and AZURE_OPENAI_RESOURCE_URL:
        # Option 2: Auto-create Knowledge Base from index
        print(f"✅ Auto-creating Knowledge Base from index: {INDEX_NAME}")
        return AzureAISearchContextProvider(
            endpoint=SEARCH_ENDPOINT,
            index_name=INDEX_NAME,
            api_key=SEARCH_API_KEY,
            credential=AzureCliCredential() if not SEARCH_API_KEY else None,
            mode="agentic",
            azure_openai_resource_url=AZURE_OPENAI_RESOURCE_URL,
            AZURE_AI_MODEL_DEPLOYMENT_NAME=MODEL_DEPLOYMENT,
            # Retrieval configuration
            knowledge_base_output_mode="extractive_data",
            retrieval_reasoning_effort="minimal",
            top_k=5,  # Number of results to retrieve
        )
    else:
        raise ValueError(
            "Configure either AZURE_SEARCH_KNOWLEDGE_BASE_NAME or both "
            "AZURE_SEARCH_INDEX_NAME and AZURE_OPENAI_ENDPOINT in your .env file"
        )

# Test provider creation
try:
    test_provider = create_search_provider()
    print("\n✅ Search provider configuration validated!")
except ValueError as e:
    print(f"\n⚠️ Configuration needed: {e}")

## Underwriting Agent Instructions 📋

We define comprehensive instructions for our loan underwriting agent that:
- Specializes in underwriting policy analysis and eligibility assessment
- Uses retrieved context for accurate criteria interpretation
- Includes appropriate regulatory disclaimers

In [ ]:
AGENT_INSTRUCTIONS = """
You are a Loan Underwriting Research Assistant with expertise in mortgage and lending guidelines.

## Your Capabilities:
- Analyze underwriting criteria including DTI ratios, LTV limits, and credit requirements
- Explain documentation requirements for various loan types
- Compare eligibility criteria across conventional, FHA, VA, and jumbo loans
- Synthesize information across multiple underwriting policy documents

## Guidelines:
1. **Use Context**: Base your answers on the retrieved underwriting policy information from the knowledge base
2. **Be Precise**: Cite specific ratios, thresholds, or requirements from the sources
3. **Acknowledge Gaps**: If underwriting criteria is not available, clearly state this
4. **Professional Tone**: Maintain a helpful, analytical communication style

## Required Disclaimers:
- Always include: "This is for informational and training purposes only."
- Recommend consulting official underwriting guidelines for actual loan decisions
- Note that underwriting criteria may vary by institution and are subject to change

## Response Format:
- Start with a direct answer to the question
- Provide supporting details from retrieved underwriting policies
- End with relevant caveats or recommendations
"""

print("📝 Agent Instructions Configured")
print(f"   Length: {len(AGENT_INSTRUCTIONS)} characters")

## Run the Underwriting Research Agent 🚀

Now we create and run our loan underwriting agent with the Azure AI Search context provider in agentic mode.

In [ ]:
async def run_underwriting_agent():
    """Run the loan underwriting research agent with agentic search."""
    
    print("=" * 60)
    print("📋 Loan Underwriting Research Agent")
    print("   Using Azure AI Search with Agentic Mode")
    print("=" * 60 + "\n")
    
    # Create the search context provider
    search_provider = create_search_provider()
    
    async with (
        search_provider,
        AzureAIAgentClient(
            project_endpoint=PROJECT_ENDPOINT,
            AZURE_AI_MODEL_DEPLOYMENT_NAME=MODEL_DEPLOYMENT,
            credential=AzureCliCredential(),
        ) as client,
        ChatAgent(
            chat_client=client,
            name="underwriting-agent",
            instructions=AGENT_INSTRUCTIONS,
            context_provider=search_provider,
        ) as agent,
    ):
        print(f"✅ Agent created: {agent.name}")
        print(f"🔍 Search mode: Agentic (multi-hop reasoning)")
        print("\n" + "-" * 60 + "\n")
        
        # Process each underwriting query
        for i, query in enumerate(UNDERWRITING_QUERIES, 1):
            print(f"📊 Query {i}: {query}")
            print("\n💬 Agent Response:")
            
            # Stream the response for better UX
            async for chunk in agent.run_stream(query):
                if chunk.text:
                    print(chunk.text, end="", flush=True)
            
            print("\n\n" + "-" * 60 + "\n")
        
        print("✅ Underwriting analysis complete!")

## Execute the Agent 🎯

Run the loan underwriting research agent. The agent uses the `underwriting-index` which contains underwriting policies, eligibility criteria, DTI/LTV requirements, and documentation guidelines.

In [ ]:
# Run the loan underwriting research agent
await run_underwriting_agent()

## Interactive Query Mode 💬

Use this cell for custom underwriting research queries:

In [ ]:
async def ask_underwriting_question(question: str):
    """Ask a custom loan underwriting research question."""
    
    search_provider = create_search_provider()
    
    async with (
        search_provider,
        AzureAIAgentClient(
            project_endpoint=PROJECT_ENDPOINT,
            AZURE_AI_MODEL_DEPLOYMENT_NAME=MODEL_DEPLOYMENT,
            credential=AzureCliCredential(),
        ) as client,
        ChatAgent(
            chat_client=client,
            name="underwriting-advisor",
            instructions=AGENT_INSTRUCTIONS,
            context_provider=search_provider,
        ) as agent,
    ):
        print(f"🤔 Question: {question}\n")
        print("💬 Response:")
        
        async for chunk in agent.run_stream(question):
            if chunk.text:
                print(chunk.text, end="", flush=True)
        print("\n")

# Example: Ask a custom question
# await ask_underwriting_question("What are the maximum LTV ratios for investment property loans?")

## Key Takeaways 📚

### Azure AI Search Context Provider

```python
from agent_framework.azure import AzureAISearchContextProvider

# Using existing Knowledge Base (recommended)
search_provider = AzureAISearchContextProvider(
    endpoint=search_endpoint,
    api_key=api_key,  # Optional with managed identity
    mode="agentic",
    knowledge_base_name="my-knowledge-base",
    knowledge_base_output_mode="extractive_data",
    retrieval_reasoning_effort="minimal",
)

# Auto-create KB from index
search_provider = AzureAISearchContextProvider(
    endpoint=search_endpoint,
    index_name="my-index",
    mode="agentic",
    azure_openai_resource_url="https://myopenai.openai.azure.com",
    AZURE_AI_MODEL_DEPLOYMENT_NAME="gpt-4o",
)
```

### Agentic vs Semantic Mode

| Aspect | Agentic Mode | Semantic Mode |
|--------|-------------|---------------|
| **Query Planning** | Multi-hop reasoning | Single query |
| **Accuracy** | ~36% improvement | Good for simple queries |
| **Speed** | Slightly slower | Faster |
| **Token Usage** | Higher | Lower |
| **Best For** | Complex analysis | Simple lookups |

### FSI Best Practices for Underwriting

1. **Always Include Disclaimers**: Underwriting decisions require official policy adherence
2. **Use Extractive Mode**: For auditability, use `knowledge_base_output_mode="extractive_data"`
3. **Minimal Reasoning**: Start with `retrieval_reasoning_effort="minimal"` for faster responses
4. **Source Attribution**: Ensure responses cite the source policy documents for compliance

### Environment Variables Needed

```env
# Required
AI_FOUNDRY_PROJECT_ENDPOINT=https://your-project.services.ai.azure.com/...
AZURE_AI_MODEL_DEPLOYMENT_NAME=gpt-4o
AZURE_AI_SEARCH_ENDPOINT=https://your-search.search.windows.net

# Option 1: Existing Knowledge Base
AZURE_SEARCH_KNOWLEDGE_BASE_NAME=my-knowledge-base

# Option 2: Auto-create from index
AZURE_SEARCH_INDEX_NAME=underwriting-index
AZURE_OPENAI_ENDPOINT=https://your-openai.openai.azure.com/

# Optional
AZURE_AI_SEARCH_API_KEY=your-api-key  # If not using managed identity
```

⚠️ **Disclaimer**: This notebook is for educational and training purposes. All underwriting scenarios are simulated. Always follow your institution's official underwriting guidelines and regulatory requirements for actual loan decisions.